In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings 
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import roc_curve, auc


In [27]:
df = pd.read_csv('../data/df.csv',sep = ',')

In [28]:
df['arrete'] = pd.to_datetime(df['arrete'])

# Encoding

In [29]:
df['Motif'] = df['Motif'].apply(lambda x : 3 if x == 'Motif tres mauvais' else 2 if x == 'Motif mauvais' else 1)

df['Situation_fam'] = df['Situation_fam'].apply(lambda x : 2 if x == 'Mauvaise situation' else 1)

df['Nature'] = df['Nature'].apply(lambda x : 2 if x == 'Mauvaise nature' else 1)

df['Solution_echec'] = df['Solution_echec'].apply(lambda x : 2 if x == 'Mauvaise solution' else 1)

df['Nature_op'] = df['Nature_op'].apply(lambda x : 2 if x == 'Mauvaise nature' else 1)

df['MTF'] = df['MTF'].apply(lambda x : 2 if x == 'Mauvais MTF' else 1)

df['ETAT_CIVIL'] = df['ETAT_CIVIL'].apply(lambda x : 2 if x == 'Mauvais etat' else 1)

df['CSP'] = df['CSP'].apply(lambda x : 2 if x == 'Mauvaise CSP' else 1)

# Logits

In [31]:
df_validation = df[df['arrete']>"2020-06-30"]

data = df[df['arrete']<"2019-12-31"]

In [32]:
# Sélectionner les données
X_train = data.copy() # On drop les var qui ont trop de NaN
X_test = df_validation.copy()

X_train = X_train.select_dtypes(exclude = 'datetime')
X_train.dropna(inplace = True)

y_train = X_train['DET']


X_train.drop(columns = 'DET', inplace = True)

X_test = X_test.select_dtypes(exclude = 'datetime')
X_test.dropna(inplace = True)

y_test = X_test['DET']

X_test.drop(columns = 'DET', inplace = True)

In [33]:
X_train.drop(columns = ['cd_op'], inplace = True)

X_test.drop(columns = ['cd_op'], inplace = True)

In [34]:
from sklearn.metrics import roc_curve, auc

liste_var = []
for col in data.drop(columns =['DET','arrete','cd_op']).columns:
    model = LogisticRegression()
    
    model.fit(X_train[[col]], y_train)
    
    y_prob = model.predict_proba(X_test[[col]])[:, 1]
    
    auc = roc_auc_score(y_test, y_prob)
    
    y_pred = model.predict(X_test[[col]])
    
    conf_matrix = confusion_matrix(y_test, y_pred)
    classification_rep = classification_report(y_test, y_pred)
    
    if auc>0.55:
        liste_var.append(col)
        print(f"{col} : {auc}")


Motif : 0.7198846737000038
niv_gravite_mtf_dfs : 0.5552407811453504
MTF : 0.5843885980343989
nb_empr_ope : 0.5601252875526606


In [35]:
liste_var

['Motif', 'niv_gravite_mtf_dfs', 'MTF', 'nb_empr_ope']

In [36]:
df[liste_var].corr()

,Motif,niv_gravite_mtf_dfs,MTF,nb_empr_ope
Motif,1.000000,0.022115,0.321923,-0.107616
niv_gravite_mtf_dfs,0.022115,1.000000,0.143122,0.168677
MTF,0.321923,0.143122,1.000000,-0.033308
nb_empr_ope,-0.107616,0.168677,-0.033308,1.000000


In [37]:
import xgboost as xgb

liste_var = []
for col in data.drop(columns =['DET','arrete','cd_op']).columns:
    params = {'booster': 'dart', 'max_depth': 9, 'eta': 2.7274786657929362e-05, 'gamma': 0.0008432711321177537, 'colsample_bytree': 0.5556056049355307, 'subsample': 0.674439847205192, 'n_estimators': 63}
    
    # Créer le modèle avec les hyperparamètres spécifiés
    model = xgb.XGBClassifier(**params)
    
    # Entraîner le modèle
    model.fit(X_train[[col]], y_train)
    
    # Faire des prédictions sur l'ensemble de test
    y_prob = model.predict_proba(X_test[[col]])[:, 1]
    

    auc = roc_auc_score(y_test, y_prob)

    print(f"{col} : {auc}")

Motif : 0.7198846737000038
niv_gravite_mtf_dfs : 0.5552407811453504
MTF : 0.5843885980343989
dls_mep_entree_def : 0.5009248791236915


KeyboardInterrupt: 

['nb_bienHOP',
 'MT_CHA_PIP',
 'nb_empr_ope',
 'niv_gravite_mtf_dfs',
 'MT_CHA_HORS_OPE_PFI',
 'MT_REV_PFI',
 'Motif',
 'MTF']